In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_redis import RedisVectorStore

from langchain_chroma import Chroma

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

In [ ]:
# Vector Store

# Load data from file
loader = TextLoader('data/sample_data.txt')
documents = loader.load()

# Split data into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
texts = text_splitter.split_documents(documents)

# Embed data
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",
    google_api_key=os.getenv('GOOGLE_API_KEY')
)

# Redis Vector Store
vector_store = RedisVectorStore(
    redis_url= os.getenv('REDIS_URL'),
    index_name="langchain_index",
    embeddings=embeddings,
    index_type="hnsw"
)


vector_store.add_documents(texts)
print("Successfully added documents to Redis vector store")

# Khusus yang pakai chroma 
# Chroma Vector Store
chroma_vector_store = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory="chroma_db"
)
print("Successfully added documents to Chroma vector store")

Successfully added documents to Redis vector store
Successfully added documents to Chroma vector store


In [8]:
vector_store.similarity_search("Mario",k=2)

[Document(metadata={}, page_content='Mobile Developer\nGetasan Apps is an application used by the Getasan District in Semarang Regency to receive reports from residents.\nTechStack : Flutter, FlutterBloc, Golang, MySQL\nGithub Url :https://github.com/marioapn3/AsetGetasan\nSirekam\tApr - May 2024\nFullstack Developer\nSIREKAM is developed to streamline the system of recording and calculating Student Activity Credit Unit (SAKU) documents at Yogyakarta Health Polytechnic (Politeknik Kesehatan Yogyakarta). This application aims to facilitate efficient documentation, calculation, and management of student activity credits, ensuring organized and accessible records for both students and administrators.\nTechStack : Laravel, AlpineJS, TailwindCSS, MySQL\nApps Url : https://www.sirekampolkesyogya.com/\nRAB PDAM Semarang Regency\tSep - Oct 2024\nFullstack Developer'),
 Document(metadata={}, page_content='Mario Aprilnino Prasetyo\nSemarang, Indonesia | +62 812 4743 0546 | https://www.linkedin.c

In [9]:
chroma_vector_store.similarity_search("Mario",k=2)

[Document(id='d4ba96f3-1e2a-4d44-a121-2ea420316dc4', metadata={'source': 'data/sample_data.txt'}, page_content='Mobile Developer\nGetasan Apps is an application used by the Getasan District in Semarang Regency to receive reports from residents.\nTechStack : Flutter, FlutterBloc, Golang, MySQL\nGithub Url :https://github.com/marioapn3/AsetGetasan\nSirekam\tApr - May 2024\nFullstack Developer\nSIREKAM is developed to streamline the system of recording and calculating Student Activity Credit Unit (SAKU) documents at Yogyakarta Health Polytechnic (Politeknik Kesehatan Yogyakarta). This application aims to facilitate efficient documentation, calculation, and management of student activity credits, ensuring organized and accessible records for both students and administrators.\nTechStack : Laravel, AlpineJS, TailwindCSS, MySQL\nApps Url : https://www.sirekampolkesyogya.com/\nRAB PDAM Semarang Regency\tSep - Oct 2024\nFullstack Developer'),
 Document(id='8d29034c-ccde-4e6b-be8a-3d56bbd17521',

In [10]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=os.getenv('GOOGLE_API_KEY')
)


In [11]:
llm.invoke("Siapa mario?")

AIMessage(content='Mario adalah karakter fiksi ikonik dan maskot dari Nintendo.  Dia adalah seorang tukang ledeng Italia yang terkenal karena petualangannya dalam video game, terutama seri *Super Mario Bros.*  Dia biasanya digambarkan sebagai seorang pria pendek, gemuk, dengan pakaian biru dan merah, dan memiliki kemampuan untuk melompat tinggi, mengumpulkan koin, dan mengalahkan musuh-musuh seperti Bowser.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--42c322a1-0808-4800-9890-1dc5c83e5930-0', usage_metadata={'input_tokens': 3, 'output_tokens': 84, 'total_tokens': 87})

In [15]:
template = """
Anda adalah bot yang bertugas menjawab pertanyaan terkair data diri Mario.
Ini adalah context yang bisa kamu gunakan : 
{context}

Pertanyaan : {question}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("user", "{question}")
])

chain = (
    {
        "context": lambda x: vector_store.similarity_search(x["question"], k=3),
        "question": lambda x: x["question"]
    }
    | prompt
    | llm
)

chain.invoke({"question": "Siapa mario?"})

AIMessage(content='Mario Aprilnino Prasetyo adalah seorang Fullstack Developer dan Mobile Developer dari Semarang, Indonesia.  Ia sedang menempuh pendidikan S1 Teknik Informatika di Universitas Dian Nuswantoro, Semarang dengan IPK 3.95/4.00 (diperkirakan lulus Januari 2026).  Ia memiliki portofolio yang mencakup pengembangan aplikasi mobile, web, dan backend menggunakan berbagai teknologi seperti Flutter, Laravel, React, Go, dan lain-lain.  Informasi kontaknya dapat diakses melalui LinkedIn, email, dan website pribadinya.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--98eb6220-4454-4e7c-8331-0a209f3f99ca-0', usage_metadata={'input_tokens': 632, 'output_tokens': 115, 'total_tokens': 747})

In [16]:
def main():
    print("\nSelamat datang! Silakan ajukan pertanyaan tentang Mario. Ketik 'exit' untuk keluar.")
    while True:
        question = input("Anda: ")
        if question.lower() == 'exit':
            print("Sampai jumpa!")
            break
        response = chain.invoke({"question": question})
        print(f"Bot: {response.content}")

if __name__ == "__main__":
    main() 


Selamat datang! Silakan ajukan pertanyaan tentang Mario. Ketik 'exit' untuk keluar.
Bot: Mario Aprilnino Prasetyo adalah seorang Mobile dan Fullstack Developer dari Semarang, Indonesia.  Ia sedang menempuh pendidikan sarjana Teknik Informatika di Universitas Dian Nuswantoro, Semarang dengan IPK 3.95/4.00 (diperkirakan lulus Januari 2026).  Ia memiliki portofolio yang meliputi pengembangan beberapa aplikasi menggunakan berbagai teknologi seperti Flutter, Golang, Laravel, React, dan lain-lain.  Anda dapat menemukan informasi lebih lanjut tentang dirinya melalui tautan LinkedIn, email, dan portfolio website yang tertera di profilnya.

Bot: Berdasarkan informasi yang diberikan, Mario Aprilnino Prasetyo tinggal di Semarang, Indonesia.

Bot: Maaf, saya tidak dapat menjawab pertanyaan Anda karena tidak ada pertanyaan yang diajukan.  Silakan ajukan pertanyaan Anda tentang data diri Mario.  Saya akan mencoba menjawabnya berdasarkan informasi yang diberikan dalam dokumen konteks.

Sampai jumpa!